In [22]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json
import pandas


In [96]:
records = []
for line in open('upload.log'):
    # evt = AciEventUpdate(1) SensorValues: sensor_id:15 proximity_ids:[12, 20, 24, 0, 0] proximity_rssi:[53, 55, 57, 0, 0] battery:194, accel:(37, 0, 144), status:0, valid_time:1501623900
    m = re.search('AciEventUpdate.*SensorValues.*sensor_id:(\d+) .*valid_time:(\d+)$', line)

    if m:
        timestamp = datetime.utcfromtimestamp(int(m.group(2))).isoformat()
        sensor_col = '%02d' % int(m.group(1))
        records.append({'ts':timestamp, sensor_col:1})
        
df = pandas.DataFrame(records)
df.set_index(['ts'])

# group by timestamp, and skip first 5 rows, as sensors are syncing up
df = df.groupby("ts").sum().iloc[5:]
df.reindex_axis(sorted(df.columns), axis=1)
df.head() # Show layout of dataframe

,02,04,12,15,20,24,42,43,47,61
ts,,,,,,,,,,
2017-08-02T00:27:40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-02T00:27:50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-02T00:28:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-02T00:28:10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-02T00:28:20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


In [94]:
# Overall participation
counts = df.sum().dropna()
overall = counts.sum() / (counts.shape[0] * df.shape[0]) * 100
print "%0.2f%% overall participation" % overall

# Individual participation
sensor_participation = counts / df.shape[0] * 100
print "-------------------------------"
print "individual sensor participation"
for (sensor_id, pct) in sensor_participation.iteritems():
    print "%2d = %6.2f%%" % (int(sensor_id), pct)

99.81% overall participation
-------------------------------
individual sensor participation
 2 =  98.32%
 4 = 100.00%
12 = 100.00%
15 = 100.00%
20 = 100.00%
24 = 100.00%
42 = 100.00%
43 = 100.00%
47 = 100.00%
